In [1]:
%run PreProcessing.ipynb

Current Progress: 100.0 %


In [2]:
summary_data_avg = summary_data_avg[summary_data_avg.inputs != 'Conventional']
summary_data_avg['harvest_timing'] = 'optimal'
summary_data_avg.loc[summary_data_avg['mean_diameter'] < 25, 'harvest_timing'] = 'early'
summary_data_avg.loc[summary_data_avg['mean_diameter'] < 30, 'harvest_timing'] = 'suitable'

<ipython-input-2-61a766c40790>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_data_avg['harvest_timing'] = 'optimal'
c:\users\tom\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
c:\users\tom\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [3]:
from sklearn.model_selection import train_test_split
X = summary_data_avg[['heat_units', 'solar_radiation', 'organic_matter', 'sand', 'silt', 'clay', 'method', 'variety', 'inputs', 'protection']]
y = summary_data_avg['harvest_timing']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 10, stratify = y)

In [4]:
print(y_train.value_counts())

suitable    378
Name: harvest_timing, dtype: int64


In [5]:
#Isolation forest algorithm to show unsupervised learning
from sklearn.ensemble import IsolationForest

print(f"The old training dataframe shape: {X_train.shape}")

#Use the algorithm for outlier detection, then use it to predict each point
clf1 = IsolationForest(max_samples=500, random_state = 10, contamination= 'auto')
preds = clf1.fit_predict(X_train)

#Any point labelled as -1 is an outlier
totalOutliers=0
for pred in preds:
    if pred == -1:
        totalOutliers=totalOutliers+1
print("Total number of outliers identified is: ",totalOutliers)

The old training dataframe shape: (378, 10)


ValueError: could not convert string to float: 'Drilled'

In [ ]:
#Remove outliers from training data.
filtered_array = []
for element in preds:
  if element != -1:
    filtered_array.append(True)
  else:
    filtered_array.append(False)
X_train = X_train[filtered_array]
y_train = y_train[filtered_array]

print(f"The new training dataframe shape: {X_train.shape}")

In [ ]:
print(y_train.value_counts())

In [ ]:
from sklearn.utils import resample

chosen_feat_list = list(X_train)

print(y_train.value_counts())

train_df = pd.merge(X_train, y_train, right_index=True, left_index=True)

early_df = train_df[train_df['harvest_timing']=='early']
suitable_df = train_df[train_df['harvest_timing']=='suitable']
optimal_df = train_df[train_df['harvest_timing']=='optimal']

n = 500

early_resampled = resample(early_df, replace=True, n_samples=n, random_state=10)
suitable_resampled = resample(suitable_df, replace=True, n_samples=n, random_state=10)
optimal_resampled = resample(optimal_df, replace=True, n_samples=n, random_state=10)

train_df_resampled = pd.concat([early_resampled, suitable_resampled, optimal_resampled], ignore_index=True)

X_train = train_df_resampled[chosen_feat_list]
y_train = train_df_resampled['harvest_timing']

print(y_train.value_counts())

In [ ]:
#create supervised classification models
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import *
from sklearn.utils.validation import check_is_fitted
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
param_grid = {'penalty': ['l1', 'l2', 'elasticnet'],  
              'alpha': [0.0001, 0.001], 
              'random_state': [10],
              'loss': ['hinge', 'modified_huber', 'perceptron']}  
   
grid_sgd = GridSearchCV(SGDClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1) 
   
# fitting the model for grid search 
grid_sgd.fit(X_train_scaled, y_train.values.ravel()) 
 
# print best parameter after tuning 
print(grid_sgd.best_params_) 
grid_sgd_preds = grid_sgd.predict(X_test_scaled) 
   
# print classification report 
print(classification_report(y_test, grid_sgd_preds, zero_division = 0)) 

conf_matrix = confusion_matrix(y_test, grid_sgd_preds)

fig, ax = plt.subplots(figsize=(8, 8))
ax.matshow(conf_matrix, cmap=plt.cm.Oranges, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Predicted', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('SGD Confusion Matrix', fontsize=18)
plt.show()

sgd = SGDClassifier(random_state = 10, alpha = 0.001, loss = 'hinge', penalty = 'l1')
sgd_model = sgd.fit(X_train_scaled, y_train.values.ravel())
sgd_model_pred = sgd_model.predict(X_test_scaled)
sgd_accuracy = accuracy_score(y_test, sgd_model_pred)

In [ ]:
param_grid = {'var_smoothing': [0.000000001, 0.00000001, 0.00000000001]}  
   
grid_gnb = GridSearchCV(GaussianNB(), param_grid, refit = True, verbose = 3,n_jobs=-1) 
   
# fitting the model for grid search 
grid_gnb.fit(X_train_scaled, y_train.values.ravel()) 
 
# print best parameter after tuning 
print(grid_gnb.best_params_) 
grid_gnb_preds = grid_gnb.predict(X_test_scaled) 
   
# print classification report 
print(classification_report(y_test, grid_gnb_preds, zero_division = 0)) 

conf_matrix = confusion_matrix(y_test, grid_gnb_preds)

fig, ax = plt.subplots(figsize=(8, 8))
ax.matshow(conf_matrix, cmap=plt.cm.Oranges, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Predicted', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('GaussianNB Confusion Matrix', fontsize=18)
plt.show()

gnb = GaussianNB(var_smoothing = 0.000000001)
gnb_model = gnb.fit(X_train_scaled, y_train.values.ravel())
gnb_model_pred = gnb_model.predict(X_test_scaled)
gnb_accuracy = accuracy_score(y_test, gnb_model_pred)

In [ ]:
param_grid = {'gamma': ['scale', 'auto'],
              'kernel': ['linear', 'poly', 'rbf']}  
   
grid_svc = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,n_jobs=-1) 
   
# fitting the model for grid search 
grid_svc.fit(X_train_scaled, y_train.values.ravel()) 
 
# print best parameter after tuning 
print(grid_svc.best_params_) 
grid_svc_preds = grid_svc.predict(X_test_scaled) 
   
# print classification report 
print(classification_report(y_test, grid_svc_preds, zero_division = 0)) 

conf_matrix = confusion_matrix(y_test, grid_svc_preds)

fig, ax = plt.subplots(figsize=(8, 8))
ax.matshow(conf_matrix, cmap=plt.cm.Oranges, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Predicted', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('SVM Confusion Matrix', fontsize=18)
plt.show()

svc = SVC(random_state = 10, gamma = 'scale', kernel = 'linear')
svc_model = svc.fit(X_train_scaled, y_train.values.ravel())
svc_model_pred = svc_model.predict(X_test_scaled)
svc_accuracy = accuracy_score(y_test, svc_model_pred)

In [ ]:
param_grid = {'random_state': [10],
              'solver': ['newton-cg', 'lbfgs'],
              'C': [1, 0.5],
              'max_iter': [250, 400]}  
   
grid_lr = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1) 
   
# fitting the model for grid search 
grid_lr.fit(X_train_scaled, y_train.values.ravel()) 
 
# print best parameter after tuning 
print(grid_lr.best_params_) 
grid_lr_preds = grid_lr.predict(X_test_scaled) 
   
# print classification report 
print(classification_report(y_test, grid_lr_preds, zero_division = 0)) 

conf_matrix = confusion_matrix(y_test, grid_lr_preds)

fig, ax = plt.subplots(figsize=(8, 8))
ax.matshow(conf_matrix, cmap=plt.cm.Oranges, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Predicted', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Logistic Regression Confusion Matrix', fontsize=18)
plt.show()

lr = LogisticRegression(random_state = 10, max_iter = 250, C = 1, solver = 'newton-cg')
lr_model = lr.fit(X_train_scaled, y_train.values.ravel())
lr_model_pred = lr_model.predict(X_test_scaled)
lr_accuracy = accuracy_score(y_test, lr_model_pred)

In [ ]:
param_grid = {'criterion': ['gini', 'entropy'],  
              'splitter': ['best', 'random'], 
              'random_state': [11],
              'max_features': [None, 'auto']}  
   
grid_dtc = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1) 
   
# fitting the model for grid search 
grid_dtc.fit(X_train_scaled, y_train.values.ravel()) 
 
# print best parameter after tuning 
print(grid_dtc.best_params_) 
grid_dtc_preds = grid_dtc.predict(X_test_scaled) 
   
# print classification report 
print(classification_report(y_test, grid_dtc_preds, zero_division = 0)) 

conf_matrix = confusion_matrix(y_test, grid_dtc_preds)

fig, ax = plt.subplots(figsize=(8, 8))
ax.matshow(conf_matrix, cmap=plt.cm.Oranges, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Predicted', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Decision Tree Confusion Matrix', fontsize=18)
plt.show()

dtc = DecisionTreeClassifier(criterion = 'gini', random_state = 11, max_features = None, splitter = 'best')
dtc_model = dtc.fit(X_train_scaled, y_train.values.ravel())
dtc_model_pred = dtc_model.predict(X_test_scaled)
dtc_accuracy = accuracy_score(y_test, dtc_model_pred)

In [ ]:
# pack the classifiers and string into a list of tuples.
ensemble_clf = VotingClassifier([("LR", lr), ("SGD", sgd), ("DTC", dtc), ("SVC", svc)], voting="hard")
ensemble_clf.fit(X_train_scaled, y_train.values.ravel())
ensemble_clf_preds = ensemble_clf.predict(X_test_scaled)
ensemble_accuracy = accuracy_score(y_test, ensemble_clf_preds)

# print classification report 
print(classification_report(y_test, ensemble_clf_preds, zero_division = 0)) 

conf_matrix = confusion_matrix(y_test, ensemble_clf_preds)

fig, ax = plt.subplots(figsize=(8, 8))
ax.matshow(conf_matrix, cmap=plt.cm.Oranges, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Predicted', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Ensemble Confusion Matrix', fontsize=18)
plt.show()

In [ ]:
#....passing it into a bagging classifier
from sklearn.ensemble import BaggingClassifier

bagging = BaggingClassifier(ensemble_clf, n_estimators=10, random_state = 10)
bagging.fit(X_train_scaled, y_train.values.ravel())
bagging_preds = bagging.predict(X_test_scaled)
bagging_accuracy = accuracy_score(y_test, bagging_preds)
   
print(classification_report(y_test, bagging_preds, zero_division = 0)) 

conf_matrix = confusion_matrix(y_test, bagging_preds)

fig, ax = plt.subplots(figsize=(8, 8))
ax.matshow(conf_matrix, cmap=plt.cm.Oranges, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Predicted', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Bagged Ensemble Confusion Matrix', fontsize=18)
plt.show()

In [ ]:
param_grid = {'criterion': ['gini', 'entropy'],  
              'class_weight': [None, 'balanced_subsample'], 
              'random_state': [10],
              'max_features': ['auto', 'sqrt', 'log2']}  
   
grid_rfc = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1) 
   
# fitting the model for grid search 
grid_rfc.fit(X_train_scaled, y_train.values.ravel()) 
 
# print best parameter after tuning 
print(grid_rfc.best_params_) 
grid_rfc_preds = grid_rfc.predict(X_test_scaled) 
   
# print classification report 
print(classification_report(y_test, grid_rfc_preds, zero_division = 0)) 

conf_matrix = confusion_matrix(y_test, grid_rfc_preds)

fig, ax = plt.subplots(figsize=(8, 8))
ax.matshow(conf_matrix, cmap=plt.cm.Oranges, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Predicted', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Random Forest Confusion Matrix', fontsize=18)
plt.show()

rfc = RandomForestClassifier(criterion = 'gini', class_weight = 'balanced_subsample', random_state = 10, max_features = 'auto')
rfc_model = rfc.fit(X_train_scaled, y_train.values.ravel())
rfc_model_pred = rfc_model.predict(X_test_scaled)
rfc_accuracy = accuracy_score(y_test, rfc_model_pred)
